In [ ]:
import datetime
import numpy as np
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum

# --- Data Models ---

class RideStatus(Enum):
    REQUESTED = "requested"
    WAITING = "waiting"
    MATCHED = "matched"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    CANCELLED = "cancelled"

@dataclass
class Location:
    latitude: float
    longitude: float
    area_name: str
    
@dataclass
class Driver:
    id: str
    current_location: Location
    acceptance_score: float  # 0-100
    is_active: bool
    last_ride_time: datetime.datetime
    wait_time_minutes: float
    reliability_score: float  # 0-100
    ride_history: List[str]  # ride_ids
    preferred_areas: List[str]
    
@dataclass
class User:
    id: str
    current_location: Location
    destination: Location
    wait_preference: int  # Maximum minutes willing to wait
    rating: float
    ride_history: List[str]  # ride_ids

@dataclass
class RideRequest:
    id: str
    user_id: str
    pickup_location: Location
    destination_location: Location
    request_time: datetime.datetime
    status: RideStatus
    estimated_distance_km: float
    estimated_duration_minutes: float
    priority_tier: int  # 0 = standard, 1 = tier 1, 2 = tier 2
    
# --- Core Pricing Logic ---

class NammaYatriPricing:
    def __init__(
        self,
        base_fare: float = 30.0,  # Base fare in INR
        per_km_rate: float = 15.0,  # Rate per km in INR
        per_minute_waiting_rate: float = 1.0,  # Rate per minute of waiting in INR
        tier1_premium: float = 15.0,  # Tier 1 premium amount in INR
        tier2_premium: float = 30.0,  # Tier 2 premium amount in INR
        demand_threshold_medium: float = 0.7,  # Threshold for medium demand (ratio of requests to drivers)
        demand_threshold_high: float = 1.2,  # Threshold for high demand
        demand_update_interval_minutes: int = 5,  # How often to update demand metrics
        max_regulatory_fare_increase: float = 0.0,  # Maximum allowed regulatory fare increase (0 for no increase)
    ):
        self.base_fare = base_fare
        self.per_km_rate = per_km_rate
        self.per_minute_waiting_rate = per_minute_waiting_rate
        self.tier1_premium = tier1_premium
        self.tier2_premium = tier2_premium
        self.demand_threshold_medium = demand_threshold_medium
        self.demand_threshold_high = demand_threshold_high
        self.demand_update_interval_minutes = demand_update_interval_minutes
        self.max_regulatory_fare_increase = max_regulatory_fare_increase
        
        # Current system state
        self.area_demand_ratios = {}  # area_name -> demand ratio
        self.area_wait_times = {}  # area_name -> average wait time in minutes
    
    def update_demand_metrics(self, active_drivers: List[Driver], pending_requests: List[RideRequest]):
        """Update demand metrics by geographic area"""
        # Group drivers and requests by area
        drivers_by_area = {}
        requests_by_area = {}
        
        for driver in active_drivers:
            area = driver.current_location.area_name
            if area not in drivers_by_area:
                drivers_by_area[area] = 0
            drivers_by_area[area] += 1
            
        for request in pending_requests:
            area = request.pickup_location.area_name
            if area not in requests_by_area:
                requests_by_area[area] = 0
            requests_by_area[area] += 1
        
        # Calculate demand ratio for each area (requests / drivers)
        for area in set(list(drivers_by_area.keys()) + list(requests_by_area.keys())):
            driver_count = drivers_by_area.get(area, 0)
            request_count = requests_by_area.get(area, 0)
            
            # Avoid division by zero
            if driver_count > 0:
                self.area_demand_ratios[area] = request_count / driver_count
            else:
                self.area_demand_ratios[area] = float('inf') if request_count > 0 else 0
                
    def calculate_base_fare(self, ride_request: RideRequest) -> float:
        """Calculate the government-mandated meter fare"""
        # Basic distance + time calculation
        distance_fare = ride_request.estimated_distance_km * self.per_km_rate
        return self.base_fare + distance_fare
    
    def get_estimated_wait_time(self, area: str, is_priority: bool = False) -> int:
        """Estimate wait time in minutes for a given area"""
        base_wait = self.area_wait_times.get(area, 5)  # Default 5 minutes if no data
        demand_ratio = self.area_demand_ratios.get(area, 1.0)
        
        # Adjust wait time based on demand
        if demand_ratio > self.demand_threshold_high:
            base_wait *= 1.5
        elif demand_ratio > self.demand_threshold_medium:
            base_wait *= 1.2
            
        # Priority reduces wait time
        if is_priority:
            base_wait *= 0.6
            
        return max(1, int(base_wait))
    
    def calculate_priority_options(self, ride_request: RideRequest) -> Dict:
        """Calculate pricing options for standard and priority tiers"""
        area = ride_request.pickup_location.area_name
        base_fare = self.calculate_base_fare(ride_request)
        demand_ratio = self.area_demand_ratios.get(area, 1.0)
        
        # Determine if we're in high demand
        is_high_demand = demand_ratio > self.demand_threshold_high
        is_medium_demand = demand_ratio > self.demand_threshold_medium
        
        # Calculate wait times
        standard_wait = self.get_estimated_wait_time(area, False)
        tier1_wait = self.get_estimated_wait_time(area, True)
        tier2_wait = max(1, int(tier1_wait * 0.7))
        
        # Adjust premiums based on demand
        tier1_amount = self.tier1_premium
        tier2_amount = self.tier2_premium
        
        if is_high_demand:
            tier1_amount *= 1.2
            tier2_amount *= 1.2
        
        # Create options
        options = {
            "standard": {
                "fare": base_fare,
                "estimated_wait_minutes": standard_wait,
                "contribution": 0,
                "total": base_fare
            },
            "tier1": {
                "fare": base_fare,
                "estimated_wait_minutes": tier1_wait,
                "contribution": tier1_amount,
                "total": base_fare + tier1_amount
            },
            "tier2": {
                "fare": base_fare,
                "estimated_wait_minutes": tier2_wait,
                "contribution": tier2_amount,
                "total": base_fare + tier2_amount
            }
        }
        
        return options
    
    def allocate_driver_priority(self, drivers: List[Driver], ride_request: RideRequest) -> List[Driver]:
        """Prioritize drivers based on various fairness factors"""
        # Get basic area info
        area = ride_request.pickup_location.area_name
        is_priority_request = ride_request.priority_tier > 0
        
        # Score each driver
        driver_scores = []
        for driver in drivers:
            # Base score is acceptance score
            score = driver.acceptance_score
            
            # Adjust for wait time - drivers waiting longer get priority
            wait_time_factor = min(10, driver.wait_time_minutes) / 10
            score += wait_time_factor * 20
            
            # Adjust for trip balance - prioritize drivers who need this trip type
            # (Simplified implementation)
            trip_balance_score = 0
            if ride_request.estimated_distance_km < 3 and len(driver.ride_history) > 5:
                # This driver needs short trips
                trip_balance_score = 10
            score += trip_balance_score
            
            # Proximity would be calculated here (omitted for simplicity)
            
            # Preferred area bonus
            if area in driver.preferred_areas:
                score += 5
                
            driver_scores.append((driver, score))
            
        # Sort by score descending
        driver_scores.sort(key=lambda x: x[1], reverse=True)
        return [d[0] for d in driver_scores]
        
    def determine_fair_match(
        self, 
        ride_request: RideRequest, 
        available_drivers: List[Driver]
    ) -> Optional[Driver]:
        """Find the most appropriate driver for a ride request"""
        # Get prioritized drivers
        prioritized_drivers = self.allocate_driver_priority(available_drivers, ride_request)
        
        if not prioritized_drivers:
            return None
            
        # For priority requests, pick from top drivers
        if ride_request.priority_tier > 0:
            # Top 30% for tier 1, top 10% for tier 2
            cutoff = max(1, len(prioritized_drivers) // (3 if ride_request.priority_tier == 1 else 10))
            top_drivers = prioritized_drivers[:cutoff]
            return np.random.choice(top_drivers) if top_drivers else None
            
        # For standard requests, use a weighted random selection from all drivers
        # to ensure fair distribution
        if prioritized_drivers:
            weights = np.linspace(1.0, 0.3, len(prioritized_drivers))
            weights = weights / np.sum(weights)  # Normalize
            selected_idx = np.random.choice(len(prioritized_drivers), p=weights)
            return prioritized_drivers[selected_idx]
            
        return None

    def create_community_incentive(self, drivers: List[Driver], total_rides: int) -> Dict:
        """incentives for community-based driver distribution"""
        # Identify underserved areas
        area_coverage = {}
        for driver in drivers:
            area = driver.current_location.area_name
            if area not in area_coverage:
                area_coverage[area] = 0
            area_coverage[area] += 1
        
        # Areas with fewest drivers get incentives
        sorted_areas = sorted(area_coverage.items(), key=lambda x: x[1])
        underserved_areas = [area for area, count in sorted_areas[:3]]
        
        # Create incentive structure
        incentives = {
            "reliability_rewards": {
                "threshold": 95,  # 95% acceptance rate
                "reward": 50  # INR per day
            },
            "peak_hour_commitment": {
                "hours": ["8:00-10:00", "17:00-19:00"],
                "min_rides": 5,
                "reward": "Priority matching for next 24h"
            },
            "underserved_areas": {
                "areas": underserved_areas,
                "bonus_per_ride": 15  # INR per ride in underserved area
            }
        }
        
        return incentives


# --- Usage Demo ---

def demo_pricing_system():
    # Initialize the pricing system
    pricing = NammaYatriPricing()
    
    # Create sample locations
    koramangala = Location(12.9352, 77.6245, "Koramangala")
    indiranagar = Location(12.9719, 77.6412, "Indiranagar")
    whitefield = Location(12.9698, 77.7500, "Whitefield")
    
    # Create sample drivers
    drivers = [
        Driver(id="D1", current_location=koramangala, acceptance_score=92.5, is_active=True,
               last_ride_time=datetime.datetime.now() - datetime.timedelta(minutes=30),
               wait_time_minutes=30, reliability_score=95, ride_history=["R1", "R2", "R3"],
               preferred_areas=["Koramangala", "Indiranagar"]),
        Driver(id="D2", current_location=koramangala, acceptance_score=85.0, is_active=True,
               last_ride_time=datetime.datetime.now() - datetime.timedelta(minutes=15),
               wait_time_minutes=15, reliability_score=88, ride_history=["R4", "R5"],
               preferred_areas=["Koramangala"]),
        Driver(id="D3", current_location=indiranagar, acceptance_score=97.2, is_active=True,
               last_ride_time=datetime.datetime.now() - datetime.timedelta(minutes=5),
               wait_time_minutes=5, reliability_score=96, ride_history=["R6", "R7", "R8", "R9"],
               preferred_areas=["Indiranagar"]),
    ]
    
    # Create sample ride requests
    requests = [
        RideRequest(id="REQ1", user_id="U1", pickup_location=koramangala, destination_location=indiranagar,
                   request_time=datetime.datetime.now(), status=RideStatus.REQUESTED,
                   estimated_distance_km=5.2, estimated_duration_minutes=22, priority_tier=0),
        RideRequest(id="REQ2", user_id="U2", pickup_location=koramangala, destination_location=whitefield,
                   request_time=datetime.datetime.now(), status=RideStatus.REQUESTED,
                   estimated_distance_km=12.7, estimated_duration_minutes=45, priority_tier=1),
    ]
    
    # Update demand metrics
    pricing.update_demand_metrics(drivers, requests)
    
    # Simulate high demand in Koramangala
    pricing.area_demand_ratios["Koramangala"] = 1.5
    pricing.area_wait_times["Koramangala"] = 8
    
    # Calculate pricing options for a sample request
    sample_request = RideRequest(
        id="REQ_SAMPLE", user_id="U_SAMPLE", 
        pickup_location=koramangala, destination_location=indiranagar,
        request_time=datetime.datetime.now(), status=RideStatus.REQUESTED,
        estimated_distance_km=5.2, estimated_duration_minutes=22, priority_tier=0
    )
    
    options = pricing.calculate_priority_options(sample_request)
    
    # Print the pricing options
    print("Pricing Options:")
    for tier, details in options.items():
        print(f"{tier.title()}: ₹{details['total']:.2f} (Base fare: ₹{details['fare']:.2f}, "
              f"Optional contribution: ₹{details['contribution']:.2f})")
        print(f"  Estimated wait: {details['estimated_wait_minutes']} minutes")
    
    # Simulate driver matching for different request tiers
    standard_request = sample_request
    tier1_request = RideRequest(**{**sample_request.__dict__, "priority_tier": 1})
    tier2_request = RideRequest(**{**sample_request.__dict__, "priority_tier": 2})
   